In [3]:
import pandas as pd
import numpy as np
from ultralytics import YOLO
import cv2
import json
import os
import matplotlib.pyplot as plt
import ast

In [202]:
def convert_to_list(string):
    return ast.literal_eval(string)

with open('data/basketball-positions.json', 'r') as f:
    basketball_data = json.load(f)
player_df = pd.read_csv('player_positions.csv', converters={'bounding_boxes': convert_to_list, 'xyz_positions': convert_to_list})

In [18]:
player_model = YOLO('models/bball_detect_model.pt')

In [19]:
results = player_model('datasets/player-data/train/images/camcourt1_1512419072502_0.png')


image 1/1 c:\Users\Jeffrey\Basketball-Player-Tracking\datasets\player-data\train\images\camcourt1_1512419072502_0.png: 736x864 (no detections), 702.9ms
Speed: 7.6ms preprocess, 702.9ms inference, 0.0ms postprocess per image at shape (1, 3, 736, 864)


In [24]:
results[0].boxes.xywh

tensor([], size=(0, 4))

In [203]:
no_court_count = 0

def get_file_contents(filename):
    directories = [
        'datasets/yolo-basketball-data/train/labels/',
        'datasets/yolo-basketball-data/val/labels/'
    ]
    
    for directory in directories:
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                return file.read()
    return None

court_segment_model = YOLO('models/court_obb_model.onnx', task='obb')

def get_court_lines(filename):
    global no_court_count
    directories = [
        'datasets/yolo-basketball-data/train/images/',
        'datasets/yolo-basketball-data/val/images/'
    ]
    filename = filename[:-4] + ".png"
    for directory in directories:
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            img = cv2.imread(file_path)
            img_height, img_width, _ = img.shape
            results = court_segment_model(file_path)
            if results[0].obb.xywhr.shape[0] == 0:
                no_court_count += 1
                return None
            court = np.array(results[0].obb.xywhr[0])
            return [court[0] / img_width, court[1] / img_height, court[2] / img_width, court[3] / img_height, court[4]]
    return None

basketball_position_data = []
for timestamp in basketball_data:
    if timestamp['annotations'][0]['visible']:
        filename1 = 'camcourt1_' + str(timestamp['timestamp']) + '_' + '0.txt'
        filename2 = 'camcourt2_' + str(timestamp['timestamp']) + '_' + '0.txt'
        bounding_box = get_file_contents(filename1)
        court_lines = get_court_lines(filename1)
        if not bounding_box is None and not court_lines is None:
            box_parts = bounding_box.split()
            basketball_position_data.append({
                'box_x': float(box_parts[1]),
                'box_y': float(box_parts[2]),
                'box_width': float(box_parts[3]),
                'box_height': float(box_parts[4]),
                'court_x': float(court_lines[0]),
                'court_y': float(court_lines[1]),
                'court_w': float(court_lines[2]),
                'court_h': float(court_lines[3]),
                'court_r': float(court_lines[4]),
                'x_pos': timestamp['annotations'][0]['center'][0],
                'y_pos': timestamp['annotations'][0]['center'][1],
                'z_pos': timestamp['annotations'][0]['center'][2]
            })
        bounding_box = get_file_contents(filename2)
        court_lines = get_court_lines(filename2)
        if not bounding_box is None and not court_lines is None:
            box_parts = bounding_box.split()
            basketball_position_data.append({
                'box_x': float(box_parts[1]),
                'box_y': float(box_parts[2]),
                'box_width': float(box_parts[3]),
                'box_height': float(box_parts[4]),
                'court_x': float(court_lines[0]),
                'court_y': float(court_lines[1]),
                'court_w': float(court_lines[2]),
                'court_h': float(court_lines[3]),
                'court_r': float(court_lines[4]),
                'x_pos': timestamp['annotations'][0]['center'][0],
                'y_pos': timestamp['annotations'][0]['center'][1],
                'z_pos': timestamp['annotations'][0]['center'][2]
            })
for idx, row in player_df.iterrows():
    file_path = row['filepath']
    img = cv2.imread(file_path)
    img_height, img_width, _ = img.shape
    results = court_segment_model(file_path)
    court_lines = None
    if results[0].obb.xywhr.shape[0] == 0:
        no_court_count += 1
        court_lines = None
    else:
        court = np.array(results[0].obb.xywhr[0])
        court_lines = [court[0] / img_width, court[1] / img_height, court[2] / img_width, court[3] / img_height, court[4]]
    
    for i in range(len(row['bounding_boxes'])):
        bounding_box = row['bounding_boxes'][i]
        if not bounding_box is None and not court_lines is None:
            basketball_position_data.append({
                'box_x': float(bounding_box[0]),
                'box_y': float(bounding_box[1]),
                'box_width': float(bounding_box[2]),
                'box_height': float(bounding_box[3]),
                'court_x': float(court_lines[0]),
                'court_y': float(court_lines[1]),
                'court_w': float(court_lines[2]),
                'court_h': float(court_lines[3]),
                'court_r': float(court_lines[4]),
                'x_pos': float(row['xyz_positions'][i][0]),
                'y_pos': float(row['xyz_positions'][i][1]),
                'z_pos': float(row['xyz_positions'][i][2])
            })
basketball_position_data = pd.DataFrame(basketball_position_data)

Loading models\court_obb_model.onnx for ONNX Runtime inference...

image 1/1 c:\Users\Jeffrey\Basketball-Player-Tracking\datasets\yolo-basketball-data\train\images\camcourt1_1582906172972_0.png: 640x640 88.5ms
Speed: 15.5ms preprocess, 88.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Jeffrey\Basketball-Player-Tracking\datasets\yolo-basketball-data\val\images\camcourt1_1582906453961_0.png: 640x640 58.9ms
Speed: 4.8ms preprocess, 58.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Jeffrey\Basketball-Player-Tracking\datasets\yolo-basketball-data\train\images\camcourt2_1582906734977_0.png: 640x640 66.3ms
Speed: 0.0ms preprocess, 66.3ms inference, 6.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Jeffrey\Basketball-Player-Tracking\datasets\yolo-basketball-data\train\images\camcourt2_1582907297060_0.png: 640x640 81.5ms
Speed: 0.0ms preprocess, 81.5ms inference, 0.0ms postprocess per ima

In [204]:
print(f"Number of images with no court detected: {no_court_count}")

Number of images with no court detected: 8


In [205]:
basketball_position_data

,box_x,box_y,box_width,box_height,court_x,...,court_h,court_r,x_pos,y_pos,z_pos
0,0.364512,0.266838,0.013271,0.015411,0.493480,...,0.561694,0.123939,177.289941,669.812214,-256.679564
1,0.354024,0.402683,0.010274,0.013128,0.490774,...,0.564007,0.106092,126.558866,706.482751,-15.172560
2,0.510274,0.377854,0.012842,0.017123,0.539342,...,0.568191,3.042346,2264.216873,755.060091,-140.640071
3,0.361943,0.549658,0.014983,0.020548,0.539385,...,0.557369,3.000590,1841.323897,1081.954716,-64.657048
4,0.726027,0.348459,0.010274,0.014269,0.539789,...,0.573262,3.031340,2908.094570,658.629903,-119.536335
...,...,...,...,...,...,...,...,...,...,...,...
2545,0.312500,0.547002,0.041256,0.136143,0.480615,...,0.425181,3.073814,174.544308,745.209749,-180.000000
2546,0.293411,0.561994,0.075739,0.174230,0.480615,...,0.425181,3.073814,180.501435,849.779985,-180.000000
2547,0.378387,0.532820,0.043719,0.138574,0.480615,...,0.425181,3.073814,279.224455,659.764764,-180.000000
2548,0.629310,0.482577,0.032020,0.120746,0.480615,...,0.425181,3.073814,711.748766,330.865382,-180.000000


In [206]:
basketball_position_data.to_csv('position_features.csv', index=False)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('position_features.csv')

In [6]:
class BasketballDataset(Dataset):
    def __init__(self, df, input_columns, target_columns, transform=None):
        self.data = df[input_columns].values
        self.targets = df[target_columns].values
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.targets[idx]
        if self.transform:
            x = self.transform(x)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [7]:
input_columns = ['box_x', 'box_y', 'box_width', 'box_height', 
                 'court_x', 'court_y', 'court_w', 'court_h', 
                 'court_r']
target_columns = ['x_pos', 'y_pos', 'z_pos']

# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=86)

# Create Datasets and DataLoaders
train_dataset = BasketballDataset(train_df, input_columns, target_columns)
test_dataset = BasketballDataset(test_df, input_columns, target_columns)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
class BasketballNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(BasketballNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize the model
model = BasketballNet(input_size=len(input_columns), output_size=len(target_columns))

In [221]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [222]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}')


Epoch 1/100, Loss: 530472.4230
Epoch 2/100, Loss: 491582.4437
Epoch 3/100, Loss: 297473.7984
Epoch 4/100, Loss: 166066.4442
Epoch 5/100, Loss: 158475.4466
Epoch 6/100, Loss: 155951.5466
Epoch 7/100, Loss: 154204.5040
Epoch 8/100, Loss: 153136.1086
Epoch 9/100, Loss: 152536.9710
Epoch 10/100, Loss: 151869.9818
Epoch 11/100, Loss: 151435.2149
Epoch 12/100, Loss: 151113.7289
Epoch 13/100, Loss: 150831.3902
Epoch 14/100, Loss: 150522.8931
Epoch 15/100, Loss: 150408.3640
Epoch 16/100, Loss: 150131.8143
Epoch 17/100, Loss: 149785.6350
Epoch 18/100, Loss: 149808.9328
Epoch 19/100, Loss: 149452.1056
Epoch 20/100, Loss: 148939.6587
Epoch 21/100, Loss: 148803.3279
Epoch 22/100, Loss: 148240.0508
Epoch 23/100, Loss: 147917.9930
Epoch 24/100, Loss: 147605.0469
Epoch 25/100, Loss: 147322.3884
Epoch 26/100, Loss: 147050.6012
Epoch 27/100, Loss: 146636.1108
Epoch 28/100, Loss: 146355.1385
Epoch 29/100, Loss: 146018.1026
Epoch 30/100, Loss: 145707.6151
Epoch 31/100, Loss: 145344.3937
Epoch 32/100, Los

In [224]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)

test_loss = test_loss / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')


Test Loss: 102689.5621


In [226]:
input_tensor = torch.tensor(df.iloc[0, :-3].to_numpy(), dtype=torch.float32).unsqueeze(0)

with torch.no_grad():
    output = model(input_tensor)

predicted_xyz = output.squeeze().numpy()
predicted_xyz

array([     476.48,      474.19,     -101.48], dtype=float32)

In [227]:
torch.save(model.state_dict(), 'position_model.pth')

In [14]:
df.mean()

box_x           0.559538
box_y           0.471805
box_width       0.037166
box_height      0.100957
court_x         0.508744
court_y         0.580394
court_w         0.798212
court_h         0.452258
court_r         1.333351
x_pos         746.384808
y_pos         720.971845
z_pos        -173.253911
dtype: float64

In [27]:
import requests

In [28]:
url = 'http://127.0.0.1:5000/predict'
image_path = 'datasets/player-data/train/images/camcourt1_1512419072502_0.png'

with open(image_path, 'rb') as image_file:
    files = {'image': image_file}
    response = requests.post(url, files=files)

# Print the response from the API
print(response.json())

{'bballs': 0, 'prediction': [[2148305.5, 2136603.5, -442022.1875], [1925459.5, 1914957.875, -396169.53125], [1492678.5, 1484520.5, -307129.65625], [2160721.0, 2149044.25, -444631.71875], [1977487.0, 1966759.0, -406907.4375], [2097248.0, 2085881.25, -431551.46875], [1734791.0, 1725291.5, -356921.84375], [2404828.25, 2391760.0, -494810.375], [2076263.625, 2065011.25, -427235.34375]]}
